# Dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

# Data Creation

In [2]:
def generate_sentence(min_len=0, max_len=20, pos=True):
    if pos:
        n = np.random.randint(np.ceil(min_len / 3), np.ceil((max_len) / 3))
        sentence = n*"a" + n*"b" + n*"c"
        return sentence, len(sentence)
    else:
        n_0 = np.random.randint(0, max_len)
        n_1 = np.random.randint(0, max_len - n_0 + 1)
        n_2 = np.random.randint(min_len - n_0 - n_1, max_len - n_0 - n_1 + 1)
        sentence = n_0 * "a" + n_1 * "b" + n_2 * "c"
        return sentence, len(sentence)

def create_data(size=10000, balance=0.1, min_len=0, max_len=20, train=True):
    data = []
    sentence_lengths = []

    for i in range(int(size*balance)):
        sentence, sentence_length = generate_sentence(min_len=min_len, max_len=max_len, pos=True)
        while sentence == "aabbcc" and train:
            sentence, sentence_length = generate_sentence(min_len=min_len, max_len=max_len, pos=True)
        if not train:
            sentence, sentence_length = "aabbcc", 6
        data.append((sentence, 1))
        sentence_lengths.append(sentence_length)
    for i in range(int((size - (size*balance)))):
        sentence, sentence_length = generate_sentence(min_len=min_len, max_len=max_len, pos=False)
        data.append((sentence, 0))
        sentence_lengths.append(sentence_length)
    
    random.shuffle(data)
    average_length = sum(sentence_lengths) / len(sentence_lengths)
    return data, average_length

train_data, avg_sent_length_train = create_data(size=1000, balance=0.5, min_len=0, max_len=20, train=True)
val_data, avg_sent_length_val = create_data(size=1000, balance=0.5, min_len=0, max_len=20, train=False)
test_data, avg_sent_length_test = create_data(size=1000, balance=0.5, min_len=21, max_len=40, train=True)

print(f"Train Data Sample:\n{train_data}")
print(f"Average Sentence Length:\n{avg_sent_length_train}")
print(f"Val Data Sample:\n{val_data}")
print(f"Average Sentence Length:\n{avg_sent_length_val}")
print(f"Test Data Sample:\n{test_data}")
print(f"Average Sentence Length:\n{avg_sent_length_test}")


Train Data Sample:
[('abc', 1), ('aaaaaaaaaaab', 0), ('aaaaaaaaaaaaccccc', 0), ('aaaabbbbcccc', 1), ('', 1), ('aaaaaaaaaaaaaaaaabb', 0), ('abc', 1), ('bbbbbbbbbbbbb', 0), ('', 1), ('aaabbbccc', 1), ('aaaaabbbbbbbbbbbb', 0), ('aaaaabbbbbbbbbbbbbb', 0), ('bbbbbbbbbbbbbbbbbbb', 0), ('aaaaaaaaaaaaaab', 0), ('abbbbbbbbbbbbbbbbbbb', 0), ('abbbbbbbcccccccccc', 0), ('aaaaaabbbbbbbb', 0), ('aaaaaaaaaaaaaaabb', 0), ('aaabbbccc', 1), ('aaaaaaaaaaaaabbbccc', 0), ('abc', 1), ('aaabbbbb', 0), ('aaaaaaaaaaaaaaaa', 0), ('aaaaaabbbbbbcccccc', 1), ('abc', 1), ('aaaaaaaaaaaaaaaaaa', 0), ('aaaaaaaaaaaaaaaaabb', 0), ('aaabb', 0), ('', 1), ('abc', 1), ('aaaaaaaaaaaaaaaabcc', 0), ('aaaaabbbbbccccc', 1), ('aabbbbbbbbbbbbbbbbb', 0), ('aaaaaaaaaaaaaaaaaa', 0), ('abc', 1), ('aaaaaaabbbbbbbbbbb', 0), ('aaaaabbbbbccccc', 1), ('aaaaaabbbbbbcccccc', 1), ('abc', 1), ('', 1), ('aaaaaaaaaaaaaaaaaaab', 0), ('aaaaaaaaabbbbbbbbbbb', 0), ('aaabbbccc', 1), ('aaaaaaaaaaaaaaaabbbb', 0), ('aaaaaaaaabbbb', 0), ('aaabcccccc', 0)

# Part B: Recurrent Neural Networks

In [3]:
# Dependencies
import torch
import torch.nn as nn
import torch.optim as optim

# Encoding

In [7]:
# Encoding data
char_to_index = {'a':0, 'b':1, 'c':2}
classes = len(char_to_index)

def one_hot_encode(char, num_classes):
    one_hot = torch.zeros(classes, dtype=torch.float32)
    one_hot[char_to_index[char]] = 1.0
    return one_hot

def creat_tensors(data, max_l):
    X = []
    y = []

    for sent, label in data:
        if sent:
            encoded = [one_hot_encode(char, classes) for char in sent]
            encoded_tensor = torch.stack(encoded)
            X.append(encoded_tensor)
            y.append(label)

    padded_X = []
    for sent in X:
        padding = torch.zeros(max_l - sent.size(0), classes, dtype=torch.float32)
        padded_X.append(torch.cat([sent, padding], dim=0))

    X = torch.stack(padded_X)
    y = torch.tensor(y, dtype=torch.float32)

    return X, y

X_train, y_train = creat_tensors(train_data, max_l=40)
X_test, y_test = creat_tensors(test_data, max_l=40)
X_val, y_val = creat_tensors(val_data, max_l=40)

# Create torch dataset

In [8]:
from torch.utils.data import TensorDataset, DataLoader
import random

BATCH_SIZE = 64

# Assuming you have X_train and y_train from the original code
indices = list(range(len(X_train)))
random.shuffle(indices)

train_indices = indices[:900]
val_indices = indices[900:]

train_set = TensorDataset(X_train[:900], y_train[:900])
val_same_set = TensorDataset(X_train[:900], y_train[:900])
val_set = TensorDataset(X_val[900:], y_val[900:])
testset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_set, BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, BATCH_SIZE, shuffle=True)
val_same_loader = DataLoader(val_same_set, BATCH_SIZE, shuffle=True)
test_loader = DataLoader(testset, BATCH_SIZE, shuffle=True)

# Setup model

In [9]:
# Set device
if torch.cuda.is_available():
    device = 'cuda:0'
elif torch.backends.mps.is_available():
    device = 'mps:0'
else:
    device = 'cpu'
print('GPU State:', device)

GPU State: mps:0


# Training loop

### RNN

In [10]:
class ElmanRNN(nn.Module):
    def __init__(self, INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, OUTPUT_SIZE) -> None:
        super(ElmanRNN, self).__init__()
        self.num_layers = NUM_LAYERS
        self.hidden_size = HIDDEN_SIZE
        self.rnn = nn.RNN(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, batch_first=True)
        self.linear = nn.Linear(HIDDEN_SIZE, OUTPUT_SIZE)
        # self.classification= nn.Sequential(
        #     nn.Linear(HIDDEN_SIZE, OUTPUT_SIZE),
        #     nn.Sigmoid()
        # )
    
    def forward(self, x):
        # output, _ = self.rnn(x)
        # logits = self.classification(output[:,-1,:])
        # return torch.squeeze(logits)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]
        out = self.linear(out)
        return out

In [13]:
INPUT_SIZE = len(char_to_index)
HIDDEN_SIZE = 5
NUM_LAYERS = 1
OUTPUT_SIZE = 1
NUM_EPOCHS = 200
lr = 0.003

model = ElmanRNN(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, OUTPUT_SIZE)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

n_total_steps = len(train_loader)
for EPOCH in range(NUM_EPOCHS):
    model.train()
    e_loss = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        targets = targets.reshape(-1, 1)
        outputs = model(inputs)             
        loss = criterion(outputs, targets)  
        optimizer.zero_grad()              
        loss.backward()                     
        optimizer.step()                   
        e_loss += loss.item()
        
        if (batch_idx+1) % 10 == 0:
            print(f'Epoch [{EPOCH + 1}/{NUM_EPOCHS}], Loss: {e_loss / len(train_loader):.4f}')

Epoch [1/200], Loss: 0.4651
Epoch [2/200], Loss: 0.4590
Epoch [3/200], Loss: 0.4596
Epoch [4/200], Loss: 0.4598
Epoch [5/200], Loss: 0.4589
Epoch [6/200], Loss: 0.4628
Epoch [7/200], Loss: 0.4575
Epoch [8/200], Loss: 0.4583
Epoch [9/200], Loss: 0.4596
Epoch [10/200], Loss: 0.4612
Epoch [11/200], Loss: 0.4596
Epoch [12/200], Loss: 0.4575
Epoch [13/200], Loss: 0.4577
Epoch [14/200], Loss: 0.4588
Epoch [15/200], Loss: 0.4572
Epoch [16/200], Loss: 0.4607
Epoch [17/200], Loss: 0.4589
Epoch [18/200], Loss: 0.4596
Epoch [19/200], Loss: 0.4612
Epoch [20/200], Loss: 0.4562
Epoch [21/200], Loss: 0.4584
Epoch [22/200], Loss: 0.4597
Epoch [23/200], Loss: 0.4608
Epoch [24/200], Loss: 0.4565
Epoch [25/200], Loss: 0.4594
Epoch [26/200], Loss: 0.4585
Epoch [27/200], Loss: 0.4578
Epoch [28/200], Loss: 0.4587
Epoch [29/200], Loss: 0.4594
Epoch [30/200], Loss: 0.4575
Epoch [31/200], Loss: 0.4558
Epoch [32/200], Loss: 0.4593
Epoch [33/200], Loss: 0.4595
Epoch [34/200], Loss: 0.4611
Epoch [35/200], Loss: 0

# Evaluation loop

In [16]:
from torcheval.metrics import BinaryF1Score

def test_loop(data_loader, model, loss_fn):
    model.eval()
    SIZE = len(data_loader.dataset)
    NUM_BATCHES = len(data_loader)
    test_loss, correct = 0, 0
    metric = BinaryF1Score()

    with torch.no_grad():
        for inputs, targets in data_loader:
            targets = targets.reshape(-1, 1)
            outputs = model(inputs)
            test_loss += loss_fn(outputs, targets).item()
            correct += (torch.round(outputs) == targets).type(torch.float).sum().item()
            targets = targets.reshape(-1)
            metric.update(outputs, targets)
    
    test_loss /= NUM_BATCHES
    correct /= SIZE

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, F1-score: {metric.compute()}, Avg loss: {test_loss:>8f} \n")

loss_fn = nn.BCELoss()
test_loop(val_loader, model, loss_fn)

ValueError: input should be a one-dimensional tensor for binary f1 score, got shape torch.Size([64, 1]).